In [ ]:
import torch
from CAETomo.cae import cae
from tkinter import Tk
import tkinter.filedialog as tkf

root = Tk()
root.attributes('-topmost', True)  # Display the dialog in the foreground.
root.iconify()

In [ ]:
file_adr = tkf.askopenfilenames()
print(*file_adr, sep="\n")

In [ ]:
CAE = cae(file_adr, dat_dim=3, dat_unit='eV', cr_range=[1.0, 3.56, 0.01], dat_scale=1.0, rescale=False, DM_file=True)

In [ ]:
CAE.make_input(min_val=0.0, max_normalize=True, rescale_0to1=False, final_dim=1)

In [ ]:
if torch.cuda.is_available():
    print("%d gpu available"%(torch.cuda.device_count()))
    cuda_device = torch.device("cuda:0")
    print(torch.cuda.get_device_name(cuda_device))
    torch.cuda.set_device(cuda_device)
    print(torch.cuda.memory_summary(device=cuda_device))
else:
    cuda_device = None

In [ ]:
num_comp = 5
channels = [8, 16, 32, num_comp]
kernels = [64, 32, 16, 0]
pooling = [2, 2, 2, 2]

CAE.create_autoencoder(num_comp=num_comp, channels=channels,
                       kernels=kernels, pooling=pooling, cuda_device=cuda_device)

In [ ]:
batch_size = 6724
CAE.create_mini_batch(batch_size)

In [ ]:
CAE.training(n_epoch=10, l_rate=0.01)

In [ ]:
CAE.show_result(save_result=True, save_prefix="test")